### Notebook to perform general utility actions

#### Update the answers in a QALD file from remote endpoint

In [25]:
# imports
import sys
sys.path.append('../code/utils/')
from gen_util import update_qald_dataset

In [26]:
# Set variables
input_file_path = "../datasets/qald9plus/wikidata/qald_9_plus_test_wikidata_new.json"
output_file_path = "../datasets/qald9plus/wikidata/qald_9_plus_test_wikidata_latest.json"
kg = "Wikidata"
languages = ["en", "de", "ru", "fr", "lt", "ba", "be", "uk", "zh", "ja", "es"]

In [27]:
update_qald_dataset(input_file_path, output_file_path, languages, kg)

json file is exported to  ../datasets/qald9plus/wikidata/qald_9_plus_test_wikidata_latest.json
